In [ ]:
import cv2 as cv
import numpy as np

In [ ]:
class Detector:
    def __init__(self, fps):
        self.fps = fps
        self.T_f = self.fps * 5  # how many frames to be accumulated

        self.F_avg = 0
        self.F_sum = 0
        self.initial_stage = True
        self.initial_i = 2

        self.pre_frame = None
        self.pre_kp = None  #前一帧keypoint
        self.pre_des = None  #前一帧描述子
        self.cur_frame = None
        self.cur_kp = None
        self.cur_des = None

        # self.output_frame_i = 0
        # self.output_interval = 2  #seconds
        # self.alarm_results = []

        self.detecte_interval = 0  #frames
        self.detecte_fram_i = 0

    def pass_frame(self, frame):
        self.detecte_fram_i += 1
        if self.detecte_fram_i < self.detecte_interval:  #只在固定的帧间隔检查
            return None
        else:
            self.detecte_fram_i = 0

        kp, des = self.sift_process(frame)
        if self.cur_frame is None:
            self.cur_frame = frame
            self.cur_kp = kp
            self.cur_des = des
            return None
        else:
            self.pre_frame = self.cur_frame
            self.pre_kp = self.cur_kp
            self.pre_des = self.cur_des
            self.cur_frame = frame
            self.cur_kp = kp
            self.cur_des = des
        return self.detect()

    def sift_process(self, frame):
        sift = cv.xfeatures2d.SIFT_create()
        kp, des = sift.detectAndCompute(frame, None)
        return kp, des

    def detect(self):
        bf = cv.BFMatcher()
        matches = bf.knnMatch(self.pre_des, self.cur_des, k=2)

        # 调整ratio
        # ratio=0.4：对于准确度要求高的匹配；
        ratio1 = 0.4
        good = []
        for m1, n1 in matches:
            # 如果最接近和次接近的比值大于一个既定的值，那么我们保留这个最接近的值，认为它和其匹配的点为good_match
            if m1.distance < ratio1 * n1.distance:
                good.append(m1)

        dis = 0
        cal_num = 10
        for match in good[:cal_num]:
            dis += match.distance
        dis /= cal_num
        if dis > 100:
            print(dis)
            return 'motion'

In [ ]:
detector = Detector(1)
frame = cv.imread('./normal1.png')  # 读取lena原图
detector.pass_frame(frame)
frame = cv.imread('./normal2.png')  # 读取lena原图
detector.pass_frame(frame)
detection = detector.detect()
print(detection)

In [ ]:
cap = cv.VideoCapture('../datas/library_short.mp4')
detector = Detector(fps=int(cap.get(cv.CAP_PROP_FPS)))

while cap.isOpened():
    ret, frame = cap.read()
    if ret == False:
        break

    detection = detector.pass_frame(frame)
    # frame_show = detector.frame_preprocess(frame)
    frame_show = cv.resize(frame, (720, 720))  # 图像放缩
    font = cv.FONT_HERSHEY_SIMPLEX
    cv.putText(frame_show, detection, (10, 100), font, 0.5, (255, 255, 0), 2)
    if detection != None:
        print(detection)

    cv.imshow('frame', frame_show)
    c = cv.waitKey(1)
    if c == 27:  #ESC键
        break
cap.release()
cv.destroyAllWindows()